In [ ]:
!pip install torch

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from sklearn.metrics import f1_score, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
Custom Dataset
custom_dataset = ImageFolder(root='path/to/dataset', transform=transform)

train_size = int(0.8 * len(custom_dataset))
val_size = len(custom_dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(custom_dataset, [train_size, val_size])

trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64,
                                          shuffle=True, num_workers=2)

val_dataset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive', train=False,
                                       download=True, transform=transform)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=64,
                                         shuffle=False, num_workers=2)

In [ ]:
from skimage import data, io
myit = iter(trainloader)
print(next(myit)[0][0].size())
io.imshow(next(myit)[0][0].permute(1,2,0).numpy())

In [ ]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding='same'),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding='same'),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding='same'),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding='same'),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding='same'),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding='same'),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding='same'),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding='same'),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding='same'),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding='same'),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding='same'),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding='same'),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding='same'),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.avgpool = nn.AdaptiveAvgPool2d((7,7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 10) # 10 classes
        )

    def forward(self, x):
      x = self.features(x)
      x = self.avgpool(x)
      x = torch.flatten(x, 1)
      x = self.classifier(x)

      return x

In [ ]:
from torchsummary import summary
model = VGG16()
model.to(device)
summary(model, (3, 32, 32))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg16 = VGG16().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg16.parameters(), lr=0.001)

In [ ]:
train_losses = []
num_epochs = 10

for epoch in range(num_epochs):
  running_loss = 0.0

  for i, data in enumerate(trainloader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()

    outputs = vgg16(inputs) #vgg16
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    if i % 100 == 99:
      print(f"Epoch [{epoch + 1}/{num_epochs}], "
      f"Step [{i + 1}/{len(trainloader)}], "
      f"Loss: {running_loss / 100:.4f}")

      train_losses.append(running_loss / 100)
      running_loss = 0.0

print("Finished Training!")

In [ ]:
vgg16.eval()
val_loss = 0.0
val_predictions = []
val_targets = []

with torch.no_grad():
  for data in valloader:
    inputs, labels = data[0].to(device), data[1].to(device)
    outputs = vgg16(inputs)
    loss = criterion(outputs, labels)
    val_loss += loss.item()

    _, predicted = torch.max(outputs, 1)
    val_predictions.extend(predicted.cpu().numpy())
    val_targets.extend(labels.cpu().numpy())

val_loss /= len(valloader)
print(f"Validation Loss: {val_loss:.4f}")


plt.figure(figsize=(8, 6))
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()
plt.show()

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in valloader:
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = vgg16(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the validation set: {accuracy:.2f}%')

In [ ]:
f1 = f1_score(val_targets, val_predictions, average='macro')
print(f"F1 Score: {f1:.4f}")

In [ ]:
conf_matrix = confusion_matrix(val_targets, val_predictions)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
cm = confusion_matrix(val_predictions, val_targets)
plt.figure(figsize=(8,6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
classes = ['your', 'dataset', 'classes']
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.show()